# Fit a Gaussian Process surrogate model

Here we define a surrogate model using Gaussian Processes.
We use the GP model from ScikitLearn - we compared it to other models like GPFlow but observed better speed and better code maintenance in this model.

In [ ]:
import warnings

import chart_studio
from besos import eppy_funcs as ef, sampling
from besos.evaluator import EvaluatorEP, EvaluatorGeneric
from besos.problem import EPProblem
from chart_studio import plotly as py
from plotly import graph_objs as go
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic
from sklearn.model_selection import GridSearchCV, train_test_split

from parameter_sets import parameter_set

We begin by:
+ getting a predefined list of 7 parameters from `parameter_sets.py`
+ making these into a `problem` with electricty use as the objective
+ and making an `evaluator` using the default EnergyPlus building.

In [ ]:
parameters = parameter_set(7)
problem = EPProblem(parameters, ["Electricity:Facility"])
building = ef.get_building()
evaluator = EvaluatorEP(problem, building)

Then we get 50 samples across this design space and evaluate them.

In [ ]:
inputs = sampling.dist_sampler(sampling.lhs, problem, 5)
outputs = evaluator.df_apply(inputs)
inputs

## Train-test split

Next we split the data into a training set (80%) and a testing set (20%).

In [ ]:
train_in, test_in, train_out, test_out = train_test_split(
    inputs, outputs, test_size=0.2
)

## Hyper-parameters

Before fitting the GP model we define the set of hyperparameters we want to optimize.
Here we use \textit{3} folds in the k-fold cross validation scheme.
We select a set of Kernel functions, which must fit the characteristics of a problem - details and examples may be found in the [Kernel cookbook](https://www.cs.toronto.edu/~duvenaud/cookbook/).
Note that the parameters of the Kernel itself are [optimized during each model fitting run](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html).

In [ ]:
hyperparameters = {
    "kernel": [
        None,
        1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
        1.0 * RationalQuadratic(length_scale=1.0, alpha=0.5),
        # ConstantKernel(0.1, (0.01, 10.0))*(DotProduct(sigma_0=1.0, sigma_0_bounds=(0.1, 10.0))**2),
        1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)),
    ]
}

folds = 3

## Model fitting

Here we fit the model using these hyperparameters.

In [ ]:
gp = GaussianProcessRegressor(normalize_y=True)

clf = GridSearchCV(gp, hyperparameters, iid=True, cv=folds)

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    clf.fit(inputs, outputs)

print(f"Best performing model $R^2$ score on training set: {clf.best_score_}")
print(f"Model $R^2$ parameters: {clf.best_params_}")
print(
    f"Best performing model $R^2$ score on a separate test set: {clf.best_estimator_.score(test_in, test_out)}"
)

## Surrogate Modelling Evaluator object
We can wrap the fitted model in a BESOS `Evaluator`.

In [ ]:
def evaluation_func(ind):
    return (clf.predict([ind])[0][0],)


GP_SM = EvaluatorGeneric(evaluation_func, problem)

This has identical behaviour to the original EnergyPlus Evaluator object.
In the next cells we generate a single input sample and evaluate it using the surrogate model and EnergyPlus.

In [ ]:
sample = sampling.dist_sampler(sampling.lhs, problem, 1)
values = sample.values[0]
print(values)

In [ ]:
GP_SM(values)[0]

In [ ]:
evaluator(values)[0]

## Running a large surrogate evaluation

In [ ]:
inputs = sampling.dist_sampler(sampling.lhs, problem, 5000)
outputs = GP_SM.df_apply(inputs)
results = inputs.join(outputs)
results.head()

## Generate an idf/epJSON file with data in dataframe

Generate an idf/epJSON file with selected row of data in dataframe and save it in current directory.

In [ ]:
# generate_building(dataframe, index, filename)
evaluator.generate_building(results, 2, "output")

## Visualization

In [ ]:
chart_studio.tools.set_credentials_file(
    username="besos", api_key="Kb2G2bjOh5gmwh1Midwq"
)
df = inputs.round(3)

# generate list if dictionaries
l = list()
for i in df.columns:
    l.extend([dict(label=i, values=df[i])])

l.extend([dict(label=outputs.columns[0], values=outputs.round(-5))])

data = [
    go.Parcoords(
        line=dict(
            color=outputs["Electricity:Facility"],
            colorscale=[[0, "#D7C16B"], [0.5, "#23D8C3"], [1, "#F3F10F"]],
        ),
        dimensions=l,
    )
]

layout = go.Layout(plot_bgcolor="#E5E5E5", paper_bgcolor="#E5E5E5")

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="parcoords-basic")